In [270]:
import pandas as pd
import numpy as np
from regimeSignal import regimeSignalModel
from Miscellaneous import Plot, FetchData
from pypfopt import expected_returns, risk_models
import datetime
from dateutil import relativedelta
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [271]:
tickers = ["ABM", "ABC", "AlEX"]
data = FetchData().test_set(ticker=tickers, startDate="2014-01-01", endDate="2014-12-31")
#data.resample('1M').mean()
data.head()

...Data for ['ABM', 'ABC', 'AlEX'] from 2014-01-01 to 2014-12-31 loaded successfully


ticker,ABC,ABM,ALEX
date,,,
2014-01-02,69.89,28.24,41.51
2014-01-03,69.94,28.43,42.14
2014-01-06,69.69,28.33,42.26
2014-01-07,70.45,28.45,42.15
2014-01-08,71.14,28.28,42.15


In [272]:
regimeSignals = pd.Series(data=[item-1 for item in np.random.choice(3, len(data.resample('1M').mean()), replace=True)],index=data.resample('1M').mean().index)
regimeSignals = regimeSignals[2:]
regimeSignals

date
2014-03-31   -1
2014-04-30    1
2014-05-31    1
2014-06-30    1
2014-07-31   -1
2014-08-31    0
2014-09-30    1
2014-10-31    0
2014-11-30    0
2014-12-31    1
Freq: M, dtype: int64

In [273]:
RSModel = regimeSignalModel(regimeSignals,data)

...Data for USTREASURY/YIELD.3 from 2014-01-02 to 2014-03-31 loaded successfully
...Data for USTREASURY/YIELD.3 from 2014-02-03 to 2014-04-30 loaded successfully
...Data for USTREASURY/YIELD.3 from 2014-03-03 to 2014-05-30 loaded successfully
...Data for USTREASURY/YIELD.3 from 2014-04-01 to 2014-06-30 loaded successfully
...Data for USTREASURY/YIELD.3 from 2014-05-01 to 2014-07-31 loaded successfully
...Data for USTREASURY/YIELD.3 from 2014-06-02 to 2014-08-29 loaded successfully
...Data for USTREASURY/YIELD.3 from 2014-07-01 to 2014-09-30 loaded successfully
...Data for USTREASURY/YIELD.3 from 2014-08-01 to 2014-10-31 loaded successfully
...Data for USTREASURY/YIELD.3 from 2014-09-02 to 2014-11-28 loaded successfully
...Data for USTREASURY/YIELD.3 from 2014-10-01 to 2014-12-31 loaded successfully


In [274]:
wts=RSModel.getWeights(verbose=True)

max sharpe
Training dates 2014-01-02 00:00:00 2014-03-31 00:00:00 
Regime Signal dates 2014-03-31 00:00:00 
Risk-free rate 0.05
ticker
ABC    -0.234095
ABM     0.076497
ALEX    0.110620
dtype: float64
ticker       ABC       ABM      ALEX
ticker                              
ABC     0.032500  0.010722  0.006764
ABM     0.010722  0.041095  0.016620
ALEX    0.006764  0.016620  0.063676

{'ABC': 0.1111111111111111, 'ABM': 0.1111111111111111, 'ALEX': 0.7777777777777779}



min vol
Training dates 2014-02-03 00:00:00 2014-04-30 00:00:00 
Regime Signal dates 2014-04-30 00:00:00 
Risk-free rate 0.05
ticker
ABC     0.000645
ABM     0.186338
ALEX   -0.046989
dtype: float64
ticker       ABC       ABM      ALEX
ticker                              
ABC     0.034075  0.007763  0.003044
ABM     0.007763  0.042696  0.013308
ALEX    0.003044  0.013308  0.062829

{'ABC': 0.4809326744812549, 'ABM': 0.2969943896070222, 'ALEX': 0.2220729359117228}



min vol
Training dates 2014-03-03 00:00:00 2014-05-30 00:

In [275]:
print(np.sum(list(wts[1].values())),np.sum(list(wts[-1].values())),np.sum(list(wts[0].values())))
wts

0.9999999999999999 1.0 1.0000000000061058


{1: {'ABC': 0.5301750736878684,
  'ABM': 0.2888733623835027,
  'ALEX': 0.18095156392862877},
 -1: {'ABC': 0.4444444444444444,
  'ABM': 0.1111111111111111,
  'ALEX': 0.44444444444444453},
 0: {'ABC': 0.43318290811589827,
  'ABM': 0.3090781901203132,
  'ALEX': 0.2577389017698943}}